In [2]:
import os
import cv2
import xml.etree.ElementTree as ET

## Convert VOC label(.xml) to YOLO Farmat(.txt)

In [3]:
annotation_path = r'E:\datalong\annotations\alpha\detect\raw\annotations'
label_path = r'E:\datalong\annotations\alpha\detect\raw\labels'
image_path = r'E:\datalong\annotations\alpha\detect\raw\images'
os.makedirs(label_path, exist_ok=True)

In [4]:
classes = ['long']

In [5]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [6]:
for xml in os.listdir(annotation_path):
    xml_file = os.path.join(annotation_path, xml)
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    img_name = root.find(".//filename").text

    img = cv2.imread(os.path.join(image_path, img_name))
    img_height, img_width = img.shape[:2]

    file_name = os.path.splitext(img_name)[0]
    
    with open(os.path.join(label_path, file_name) + '.txt', 'w') as f:
        for obj in root.iter('object'):
            cls = obj.find('name').text  # 'long' or 'short'
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((img_width, img_height), b)
            f.write(str(cls_id) + ' ' + ' '.join([str(a) for a in bb]) + '\n')

    